In [42]:
import math
from collections import deque, defaultdict
import pandas as pd

In [43]:
data = pd.read_csv('toyota_dashboard_365day_dataset.csv')

In [44]:
df = pd.DataFrame(data)

In [45]:
df.columns

Index(['Date', 'Shift', 'Batch ID', 'Model', 'Dashboard Type',
       'Display Resolution', 'Backlight Brightness (nits)',
       'Touch Screen Sensitivity (ms)', 'Button Press Response (ms)',
       'Wiring Harness Check', 'CAN Bus Signal Integrity', 'Boot Time (sec)',
       'Software Version', 'Warning Light Cycle OK',
       'Touch Calibration Status', 'Tested RPM Reading',
       'Tested Speed Reading (km/h)', 'Tested Fuel Level (%)', 'Tester ID',
       'Pass/Fail'],
      dtype='object')

In [46]:
df.columns.shape

(20,)

In [47]:
df.head()

,Date,Shift,Batch ID,Model,Dashboard Type,Display Resolution,Backlight Brightness (nits),Touch Screen Sensitivity (ms),Button Press Response (ms),Wiring Harness Check,CAN Bus Signal Integrity,Boot Time (sec),Software Version,Warning Light Cycle OK,Touch Calibration Status,Tested RPM Reading,Tested Speed Reading (km/h),Tested Fuel Level (%),Tester ID,Pass/Fail
0,2025-01-01,Shift 1,DASH8607,Camry,Hybrid,1024x600,442.5,33.22,31.14,Fail,Fail,8.13,v1.27.1,Yes,Not Calibrated,2186.0,31.0,71.2,EMP771,Pass
1,2025-01-01,Shift 1,DASH4223,Highlander,Hybrid,800x480,340.5,14.59,24.45,Pass,Pass,7.33,v1.26.0,No,Not Calibrated,1152.0,12.0,1.9,EMP120,Pass
2,2025-01-01,Shift 1,DASH9115,Tacoma,Analog,800x480,453.5,19.05,32.00,Pass,Fail,5.19,v1.27.1,No,Calibrated,5029.0,197.0,16.7,EMP916,Pass
3,2025-01-01,Shift 2,DASH3225,Tacoma,Digital,1024x600,438.4,13.18,20.06,Fail,Fail,3.97,v1.26.0,Yes,Not Calibrated,6081.0,125.0,49.8,EMP342,Pass
4,2025-01-01,Shift 2,DASH9392,RAV4,Digital,800x480,344.2,37.95,16.50,Pass,Pass,7.15,v1.25.3,No,Not Calibrated,1135.0,105.0,34.9,EMP957,Pass


In [48]:
numeric_cols = df.select_dtypes(include='number').columns.tolist()

In [49]:
numeric_cols

['Backlight Brightness (nits)',
 'Touch Screen Sensitivity (ms)',
 'Button Press Response (ms)',
 'Boot Time (sec)',
 'Tested RPM Reading',
 'Tested Speed Reading (km/h)',
 'Tested Fuel Level (%)']

In [50]:
fields = numeric_cols

### **Problem 1: Rolling Brightness Average**

**Purpose**: Use `deque(maxlen=3)` to track the last 3 brightness readings for each Dashboard Type.

> For each **Dashboard Type**, compute the **rolling average of Backlight Brightness** per new entry.

**Expected Output**:
A table showing for each row:

* Dashboard Type
* Current Brightness
* Rolling Brightness Avg (last 3)

In [51]:
rolling_windows = defaultdict(lambda: deque(maxlen=3))

In [52]:
for idx, row in df.iterrows():
    dashboard_type = row['Dashboard Type']
    brightness = row['Backlight Brightness (nits)']
    
    rolling_windows[dashboard_type].append(brightness)
    
        
    if len(rolling_windows[dashboard_type]) == 3:
        window = rolling_windows[dashboard_type]
        avg = sum(window) / len(window)
    
        print(f'{dashboard_type} -> Window: {list(window)} -> Avg: {avg:.2f}')

Digital -> Window: [438.4, 344.2, 419.4] -> Avg: 400.67
Analog -> Window: [453.5, 396.3, 389.0] -> Avg: 412.93
Digital -> Window: [344.2, 419.4, 494.4] -> Avg: 419.33
Digital -> Window: [419.4, 494.4, 202.9] -> Avg: 372.23
Hybrid -> Window: [442.5, 340.5, 431.4] -> Avg: 404.80
Digital -> Window: [494.4, 202.9, 404.5] -> Avg: 367.27
Digital -> Window: [202.9, 404.5, 203.4] -> Avg: 270.27
Hybrid -> Window: [340.5, 431.4, 232.8] -> Avg: 334.90
Analog -> Window: [396.3, 389.0, 496.5] -> Avg: 427.27
Analog -> Window: [389.0, 496.5, 311.6] -> Avg: 399.03
Digital -> Window: [404.5, 203.4, 297.4] -> Avg: 301.77
Digital -> Window: [203.4, 297.4, 461.6] -> Avg: 320.80
Hybrid -> Window: [431.4, 232.8, 281.9] -> Avg: 315.37
Digital -> Window: [297.4, 461.6, 493.7] -> Avg: 417.57
Hybrid -> Window: [232.8, 281.9, 412.9] -> Avg: 309.20
Hybrid -> Window: [281.9, 412.9, 499.0] -> Avg: 397.93
Analog -> Window: [496.5, 311.6, 210.1] -> Avg: 339.40
Hybrid -> Window: [412.9, 499.0, 317.8] -> Avg: 409.90
Hy

### **Problem 4.2: Track Last 5 Failures by Model**

**Purpose**: Use `deque(maxlen=5)` to monitor recent failed tests for each **Model**.

> For each failure (`Pass/Fail == "Fail"`), add to a rolling queue per model.

**Expected Output**:
A summary report for each Model showing:

* Total tests
* Number of Failures
* Last 5 Batch IDs of failed tests

In [53]:
rolling_windows = defaultdict(lambda: deque(maxlen=5))

In [54]:
total_test = 0
num_failures = 0
for idx, row in df.iterrows():
    pass_fail = row['Pass/Fail']
    batch_id = row['Batch ID']
    model = row['Model']
    total_test += 1
    if pass_fail == 'Fail':
        num_failures += 1
        rolling_windows[model].append(batch_id)

print(f'Total tests: {total_test}; Number of Failures: {num_failures}')  
for model, batch_ids in rolling_windows.items():
    print(f'{model} -> last 5 failures: {list(batch_ids)}')

Total tests: 2190; Number of Failures: 87
Highlander -> last 5 failures: ['DASH8916', 'DASH8916', 'DASH8916', 'DASH8916', 'DASH7657']
Yaris -> last 5 failures: ['DASH2162', 'DASH1750', 'DASH1750', 'DASH2162', 'DASH9208']
Tacoma -> last 5 failures: ['DASH3956', 'DASH7173', 'DASH7173', 'DASH3956', 'DASH3956']


### **Problem 4.3: Touch Latency Spike Detection**

**Purpose**: Use a rolling window of 4 (`deque(maxlen=4)`) to detect a sudden increase in `Touch Screen Sensitivity (ms)`.

> Mark a spike if the latest value is **50% greater than the average** of the previous 3.

**Expected Output**:
List of rows with:

* Date, Model
* Sensitivity values in the window
* Spike detected (Yes/No)

In [55]:
rolling_windows = defaultdict(lambda: deque(maxlen=4))

In [56]:
for idx, row in df.iterrows():
    touch_screen_sensitivity = row['Touch Screen Sensitivity (ms)']
    date = row['Date']
    model = row['Model']
    
    # append AFTER check
    window = rolling_windows[model]
    
    # Only check for the spike when there are 3 values
    if len(window) == 3:
        avg_of_3 = sum(window) / 3
        if touch_screen_sensitivity > 1.5 * avg_of_3:
            print(f'{date}: {model} -> {list(window) + [touch_screen_sensitivity]} -> Spike: Yes')
    
    window.append(touch_screen_sensitivity)
    

2025-01-06: Corolla -> [10.65, 32.1, 35.11, 44.26] -> Spike: Yes


### **Problem 4.4: CAN Bus Integrity Monitor**

**Purpose**: Use a `deque(maxlen=10)` to store the last 10 signal integrity results (`Pass`/`Fail`).

> For each shift, maintain a buffer for total Passes vs. Fails.

**Expected Output**:
For each Shift:

* Total Tests
* Number of Fails (last 10)
* Warning: "Check wiring" if more than 5 fails in last 10

In [57]:
rolling_windows = defaultdict(lambda: deque(maxlen=10))

In [58]:
total_tests = 0
for idx, row in df.iterrows():
    total_tests += 1
    CAN_bus = row['CAN Bus Signal Integrity']
    shift = row['Shift']
    rolling_windows[shift].append(CAN_bus)
    
print(f'Total tests: {total_tests}')

Total tests: 2190


In [59]:
for shift, CAN_bus in rolling_windows.items():
    num_failures = sum(1 for result in CAN_bus if result == 'Fail')
    if num_failures >= 5:
        print(f'{shift} -> {list(CAN_bus)} -> Warning: Check Wiring')
    else:
        print(f'{shift} -> {list(CAN_bus)}')        

Shift 1 -> ['Fail', 'Fail', 'Pass', 'Pass', 'Pass', 'Pass', 'Fail', 'Fail', 'Pass', 'Pass']
Shift 2 -> ['Fail', 'Pass', 'Fail', 'Pass', 'Pass', 'Fail', 'Pass', 'Pass', 'Fail', 'Fail'] -> Warning: Check Wiring


### **Problem 4.5: Boot Time Stability Tracker**

**Purpose**: Use a `deque(maxlen=5)` to track Boot Time per model.

> Report if the **standard deviation** of the last 5 boot times exceeds 2 seconds.

**Expected Output**:
For each Model:

* Batch ID
* Boot Times in the window
* Std Dev
* Stability Flag (Stable/Unstable)

In [62]:
rolling_windows = defaultdict(lambda: deque(maxlen=5))

In [63]:
for idx, row in df.iterrows():
    model = row['Model']
    boot_time = row['Boot Time (sec)']
    batch_id = row['Batch ID']
    rolling_windows[model].append(boot_time)
    
    if len(rolling_windows[model]) == 5:
        mean = sum(rolling_windows[model]) / len(rolling_windows[model])
        variance = sum((x - mean) ** 2 for x in rolling_windows[model]) / len(rolling_windows[model])
        std_dev = math.sqrt(variance)
        
        if std_dev > 2:
            print(f'{model} --> {batch_id} --> {list(rolling_windows[model])} --> Standard Deviation: {std_dev} --> Unstable')
        else:
            print(f'{model} --> {batch_id} --> {list(rolling_windows[model])} --> Standard Deviation: {std_dev} --> Stable')

RAV4 --> DASH9323 --> [7.15, 3.7, 7.49, 4.54, 5.1] --> Standard Deviation: 1.480413455761599 --> Stable
Camry --> DASH7194 --> [8.13, 7.51, 5.3, 3.47, 7.22] --> Standard Deviation: 1.7125956907571618 --> Stable
Camry --> DASH9653 --> [7.51, 5.3, 3.47, 7.22, 3.36] --> Standard Deviation: 1.7695807413056912 --> Stable
Highlander --> DASH7255 --> [7.33, 2.87, 2.15, 4.94, 3.24] --> Standard Deviation: 1.854363502660684 --> Stable
Highlander --> DASH7843 --> [2.87, 2.15, 4.94, 3.24, 6.59] --> Standard Deviation: 1.6037381332374683 --> Stable
Highlander --> DASH3988 --> [2.15, 4.94, 3.24, 6.59, 4.47] --> Standard Deviation: 1.5117063206853374 --> Stable
Highlander --> DASH9080 --> [4.94, 3.24, 6.59, 4.47, 2.87] --> Standard Deviation: 1.324890938907803 --> Stable
Tacoma --> DASH1418 --> [5.19, 3.97, 9.64, 4.85, 4.88] --> Standard Deviation: 2.0085477340606075 --> Unstable
RAV4 --> DASH4570 --> [3.7, 7.49, 4.54, 5.1, 6.57] --> Standard Deviation: 1.3729238871838454 --> Stable
RAV4 --> DASH145

Great! You've uploaded a dataset containing **365 days of Toyota dashboard tests**, with various parameters such as model, resolution, backlight brightness, CAN bus signal, RPM, and more.

Below are **10 skill-building problems** using `deque`, following the same structure and techniques we practiced before (such as sliding windows, rollbacks, buffering, rolling averages, and error tracking). No new formulas or concepts are introduced unless explained thoroughly.

---



---



---



---



---



---

### 🔁 **Problem 4.6: Button Press Lag Recovery**

**Purpose**: Simulate a recovery queue for high response times (>30 ms).

> Use a `deque` as a backlog. If 3 or more slow presses happen consecutively, suggest a software patch.

**Expected Output**:
For each instance:

* Batch ID
* 3 consecutive response times
* Patch Recommended: Yes/No

---

### 🔁 **Problem 4.7: Dashboard Swap Buffer**

**Purpose**: Use `deque.appendleft()` to simulate rollback of incorrectly installed dashboards.

> If Touch Calibration is "Not Calibrated" and Warning Light Cycle is "No", queue them for re-check in reverse order.

**Expected Output**:
List of Batch IDs in the order they should be rechecked.

---

### 🔁 **Problem 4.8: RPM Spike Window**

**Purpose**: Use `deque(maxlen=5)` to analyze rolling average RPM.

> Flag if current reading is **twice the rolling average** of previous 4.

**Expected Output**:
For each spike:

* Batch ID, Model
* RPM values in the window
* Spike Flag (Yes/No)

---

### 🔁 **Problem 4.9: Dashboard Tester Buffer**

**Purpose**: For each **Tester ID**, use a `deque(maxlen=10)` to track their last 10 test results (`Pass`/`Fail`).

> Report testers with more than 3 fails in their last 10.

**Expected Output**:
Table with:

* Tester ID
* Last 10 outcomes
* Alert: Underperformance? (Yes/No)

---

### 🔁 **Problem 4.10: Hybrid Model Fuel Depletion Warning**

**Purpose**: Use `deque(maxlen=3)` to monitor low fuel levels (<10%) in Hybrid dashboards.

> If 3 consecutive Hybrid dashboards show low fuel, issue a fuel system check alert.

**Expected Output**:
Each alert row should include:

* Date, Model
* Fuel readings
* Alert Raised: Yes/No

---

Would you like the solution for a specific one next (e.g., Problem 4.1)?
